# Assignment 3
## Group name: [enter your group name with either the prefix "ID2214-" or "FID3214-"]
### Project members: 
[Enter name, email]

[Enter name, email]

[Enter name, email]

### Declaration:
By submitting this assignment, it is hereby declared that all group members listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214), no part of the solution has been provided by someone not listed as a project member above, and no part of the solution has been generated by a system.

It is furthermore declared that the submitted assignment will not be shared during the course, with any individual other than the group members listed above and teachers of the course ID2214/FID3214. In particular, the assignment will not be uploaded to any public repository. The submitted assignment can be shared after the course only if written consent has been provided by the course responsible of ID2214/FID3214.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [4]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [7]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.9.13
NumPy version: 1.21.5
Pandas version: 1.4.4
sklearn version: 1.1.1


## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    new_df = df.copy()
    marked = []
    column_names = new_df.columns.values

    for col_name in column_names:
        if col_name != 'CLASS' and col_name != 'ID':

            unique_vals = pd.unique(new_df[col_name])
            # if only missing vals or only one unique value (apart from missing values)
            if len(unique_vals) == 1 or (len(unique_vals) == 2 and np.isnan(unique_vals).any()): # only one unique value
                marked.append(col_name)

    column_filter = list(filter(lambda x: x in marked , column_names))

    filtered_df = new_df.drop(columns=column_filter)
    return filtered_df, marked


def apply_column_filter(df, column_filter):
    new_df = df.copy()
    filtered_df = new_df.drop(columns=column_filter)
    return filtered_df


def create_normalization(df, normalizationtype="minmax"): 
    new_df = df.copy()
    normalization = {}

    for col in df.columns: 
        if col in ["ID", "CLASS"]:
            continue # these classes are not to be included in normalization

        if np.issubdtype(df[col].dtype, np.floating) or np.issubdtype(df[col].dtype, np.integer): # check if data in col is of type int or float before normalizing
            if normalizationtype == "minmax": 
                col_min = float(df[col].min()) 
                col_max = float(df[col].max())

                if col_max > col_min: # avoid div by 0
                    new_df[col] = (df[col] - col_min) / (col_max - col_min)
                    # new_df[col] = (df[col] - col_min) / (col_min - col_max)
                    # new_df[col] = ((x-col_min)/(col_max - col_min) for x in df[col]) --> from the lectures but this did not work for me

            normalization[col] = ("minmax", col_min, col_max)


            if normalizationtype == "zscore":
                mean = df[col].mean()
                std = df[col].std()

                if std != 0 :
                    new_df[col] = (df[col] - mean) / std
                    # new_df[col] = df[col].apply(lambda x: (x - mean)/std)  --> from the lectures but this did not work for me
            
                normalization[col] = ("zscore", mean, std)

    return new_df, normalization


# any dataframe (with the same columns) can now use the normalization parameters to normalize the data
def apply_normalization(df, normalization) : 
    new_df = df.copy();
    # for each column we take the normalization type, the first value (ex min) and second value (ex max) and loop over all columns
    for col, (norm_type, a, b) in normalization.items() : 
        if col not in new_df.columns: 
            continue
        if norm_type == "minmax" : 
            col_min = a;
            col_max = b;
            if col_max > col_min : 
                new_df[col] = (df[col] - col_min) / (col_max - col_min)
                new_df[col] = new_df[col].clip(0,1) # limiting output range to 0 , 1 
        if norm_type == "zscore" :
            mean = a;
            std = b;
            if std != 0: 
                new_df[col] = (df[col] - mean) / std
            
    return new_df

def create_imputation(df):
    
    new_df = df.copy(deep=True)
    special_columns = {"ID", "CLASS"}
    imputation = {}

    for col in new_df.columns:

        if col in special_columns:
            continue

        t = new_df[col]

        # Handle categorical columns
        if str(t.dtype) == "category":
            if t.notna().any():
                m = t.mode(dropna=True)
                if len(m) > 0:
                    val = m.iloc[0]
            else:
                val = t.cat.categories[0] if len(t.cat.categories) > 0 else ""

            if val not in t.cat.categories:
                new_df[col] = new_df[col].cat.add_categories([val])

            imputation[col] = val
            new_df[col] = new_df[col].fillna(val)

        # Handle numeric columns
        if pd.api.types.is_numeric_dtype(t):
            mean_val = t.mean(skipna=True)
            if pd.isna(mean_val):
                mean_val = 0
            imputation[col] = mean_val
            new_df[col] = t.fillna(mean_val)

        # Handle object (string) columns
        if t.dtype == object:
            if t.notna().any():
                m = t.mode(dropna=True)
                val = m.iloc[0] if len(m) > 0 else ""
            else:
                val = ""
            imputation[col] = val
            new_df[col] = t.fillna(val)

    return new_df, imputation

def apply_imputation(df, imputation):

    new_df = df.copy(deep=True)

    for col, val in imputation.items():
        if col not in new_df.columns:
            continue

        t=new_df[col]

        if str(t.dtype) == "category":
            if val not in t.cat.categories:
                new_df[col] = t.cat.add_categories([val])

        new_df[col] = new_df[col].fillna(val)

    return new_df


def create_bins(df, nobins=10, bintype="equal-width"):
    new_df = df.copy()
    binning = {}
    dtypes = new_df.dtypes.to_dict()
    for col_name, col_type in dtypes.items():
        # case if numerical and not class/id
        if col_name not in ("CLASS", "ID") and col_type in (float, int):
            if bintype=="equal-width":
                res, bins = pd.cut(new_df[col_name], bins=nobins, retbins=True, labels=False, duplicates="drop")
            else:
                res, bins = pd.qcut(new_df[col_name], nobins, retbins=True, labels=False, duplicates="drop")
            
            # Set the categories of the discretized features to be [0,...,nobins-1]
            categories = list(range(0, nobins))

            # Change the first and the last element of each binning to -np.inf and np.inf respectively 
            bins[0] = -np.inf
            bins[-1] = np.inf
            binning[col_name] = bins

            new_df[col_name] = pd.Categorical(res, categories=categories)
        else:
            # else, we just want them to be made to type category
            new_df[col_name] = new_df[col_name].astype("category")

    return new_df, binning

def apply_bins(df, binning):
    new_df = df.copy()

    column_names = new_df.columns.values
    bin_col_names = binning.keys()
    # for col_name, bins in binning.items():
    for col_name in column_names:
        if col_name in bin_col_names:
            bins = binning[col_name]
            new_res = pd.cut(new_df[col_name], bins=bins, labels=False, include_lowest=True)
            # Set the categories of the discretized features to be [0,...,nobins-1]
            categories = list(range(0, len(bins)-1))
            new_df[col_name] = pd.Categorical(new_res, categories=categories)
        else:
            # Set all columns in the new dataframe to be of type "category"
            new_df[col_name] = new_df[col_name].astype("category")
    return new_df
   

def create_one_hot(df):
    new_df = df.copy()
    one_hot = {}

    for col in df.columns : 
        if col in ["ID", "CLASS"]:
            continue 
        
        if df[col].dtype == "object" or pd.api.types.is_categorical_dtype(df[col]): # (hint 2) check for obj and category columns
            
            categories = sorted(df[col].dropna().unique()) # list categories in this col --> then 
            one_hot[col] = categories # save these categories to the one_hot array

            dummies = pd.get_dummies(df[col], prefix=col, dtype=float) # create new binary columns for each category

            new_df = new_df.drop(columns=[col]) # Drop the original column  
            new_df = new_df.join(dummies) # and then add the dummy columns
    
    return new_df, one_hot


def apply_one_hot(df, one_hot) : 
    new_df = df.copy()

    for col, categories in one_hot.items() :
        for cat in categories : 
            new_col = f"{col}_{cat}" # new name of of column name and categoarial value (hint 3)
            new_df[new_col] = (new_df[col] == cat).astype(float) # new columns as type float (hint 4)

        new_df = new_df.drop(columns=[col]) # drop the original column when we don't need it anymore

    return new_df

def split(df, testfraction = 0.5):
    random_indices = np.random.permutation(df.index)
    test_size = int(testfraction * len(df))

    train_indices = random_indices[test_size:]
    test_indices = random_indices[:test_size]

    test_df = df.loc[test_indices].copy()
    training_df = df.loc[train_indices].copy()

    return training_df, test_df


def accuracy(df, correctlabels):
    new_df = df.copy()
    div = len(correctlabels)

    new_df['Max'] = new_df.idxmax(axis=1)
    # display('df with max', df)
    # now count how many times
    count = 0
    for i in range(div):
        val = correctlabels[i]
        if val == new_df['Max'][i]:
            count+=1
    return count / div


def folds(df, nofolds) :
    folds = [] # where the folds will go later

    shuffled_indexes = np.random.permutation(df.index) # random premutation creating a shuffled df
    foldslist = np.array_split(shuffled_indexes, nofolds) # split the shuffled indexes into equal sized arrays

    for index in foldslist : 
        fold_df = df.loc[index] 
        fold_df = fold_df.reset_index(drop=True) 
        folds.append(fold_df) 
    
    return folds

def brier_score(df, correctlabels):

    # Check that the number of correct labels matches the number of rows in the dataframe
    if len(correctlabels) != len(df):
        raise ValueError("Each row in the dataframe must have one corresponding correct label!")

    tot_error = 0

    # Iterate through each prediction and its corresponding true label
    for i, label in enumerate(correctlabels):
        correct_vector = np.zeros(len(df.columns))
        correct_index = np.where(df.columns == label)[0][0]
        correct_vector[correct_index] = 1.0
        
        predicted_vector = df.iloc[i].values
        # Compute squared error between predicted and true probability vectors
        tot_error += np.sum((predicted_vector - correct_vector) ** 2)
    
    return tot_error / len(df)


def auc(df, correctlabels):
    cols = df.columns.values

    total_auc = 0
    for c in cols:
        # calculate binary auc for each class label
        df_scores = df[c]

        # scores of true positives, scores of false postiive, using dictionary
        scores = {}
        for i in range(len(df_scores)):
            i_score = df_scores[i]
            i_true = correctlabels[i]
            if i_score not in scores:
                scores[i_score] = [0,0] 
            if i_true == c: # true positive
                scores[i_score][0] +=1
            else: # false positive
                scores[i_score][1] +=1

        # reverse sorted on the scores and iterate additively to calculate AUC
        reverse_scores = sorted(scores.items(), key=lambda item: item[0], reverse=True)
        tp = np.array([v[0] for _, v in reverse_scores])
        fp = np.array([v[1] for _, v in reverse_scores])

        Tot_tp = tp.sum()
        Tot_fp = fp.sum()

        if Tot_tp == 0 or Tot_fp == 0:
            continue

        # based on lecture slides
        temp_auc = 0.0
        Cov_tp = 0.0

        for i in range(len(tp)):
            if fp[i] == 0: # num of false positives is 0
                Cov_tp += tp[i]
            elif tp[i] == 0: # num of true positives is 0
                temp_auc += (Cov_tp / Tot_tp) * (fp[i] / Tot_fp)
            else: # both non-zero
                temp_auc += (Cov_tp / Tot_tp) * (fp[i] / Tot_fp) + (tp[i] / Tot_tp) * (fp[i] / Tot_fp) / 2
                Cov_tp += tp[i]
        weight = Tot_tp / df.shape[0]
        # print(f"weight of {c}, {weight}")
        total_auc += temp_auc * weight
    return total_auc


## 1. Define the class RandomForest

In [3]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 



In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.13 s.
Testing time: 0.04 s.
Accuracy: 0.9019
AUC: 0.9928
Brier score: 0.1767


In [5]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0204


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [6]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.15 s.
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9741
Brier score: 0.0975


## 2b. Estimate predictive performance using out-of-bag predictions

In [8]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0

In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.96 s.
OOB accuracy: 0.9555
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9718
Brier score: 0.0986


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.